In [1]:
import numpy as np
import torch
from torchinfo import summary

import modules.dataloaders as dataloaders
import modules.brevitas.model_mobilenetv2_mini_Resnet_Sparse24_Brevitas as cnv_model

import modules.metrics as metrics
import modules.val_final_model as val_final_model
import modules.utils as utils

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


# Load Model

In [2]:
model = cnv_model.MobileNetV2_MINI_RESNET_SPARSE24().to('cuda')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/nn/mixin/base.py:77: UserWarning: Keyword arguments are being passed but they not being used.
  warn('Keyword arguments are being passed but they not being used.')


### Torchinfo

In [3]:
print(summary(model, input_size=(1, 3, 224, 224)))

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Layer (type:depth-idx)                                                           Output Shape              Param #
MobileNetV2_MINI_RESNET_SPARSE24                                                 [1, 2]                    --
├─Sequential: 1-1                                                                [1, 64, 14, 14]           59,822
│    └─QuantIdentity: 2-1                                                        [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                       [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                       [1, 3, 224, 224]          1
├─Sequential: 1-72                                                               --                        (recursive)
│    └─QuantReLU: 2-88                                                           --                        (recursive)
│    │    └─ActQuantProxyFromInjector: 3-96                                      --           

# Load Weights

In [4]:
weights_folder = './experiments_brevitas/test_v61_mini_resnet_sparsefirst24_full_ds/weights'
model_ckpnt = weights_folder + '/Mobilenetv2_Mini_Resnet_Sparse24_classifier__best_mean_F1.pt'

In [5]:
utils.load_checkpoint(
    model_path = model_ckpnt, 
    model = model,
    device = 'cuda'
)

Loading Model. Trained during 112 epochs


112

In [6]:
model.to('cpu')
model.eval();

# Test with DFire MINI Dataset: Train and Test

In [7]:
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Test DFire MINI

In [8]:
with torch.no_grad():
    val_metrics = val_final_model.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        device='cpu')

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 44.42it/s]

SMOKE -> Precision: 1.0000 - Recall: 0.9091 - Accuracy: 0.9333 - F1: 0.9524
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


In [9]:
print('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
print(val_metrics)


Testing with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT
{'Accuracy': [0.9333333373069763, 1.0], 'Precision': [1.0, 1.0], 'Recall': [0.9090909361839294, 1.0], 'F1': [0.9523809552192688, 1.0]}


# Test with DFire Full Test

In [10]:
test_dfire_loader = dataloaders.get_dfire_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306


In [ ]:
with torch.no_grad():
    val_metrics = val_final_model.eval_fn(
        loader=test_dfire_loader, 
        model=model,                         
        device='cpu')

Validating:  82%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                        | 56/68 [01:10<00:14,  1.21s/it]